In [12]:
import pandas as pd
import re

In [3]:
js = pd.read_csv('10k.csv')

In [4]:
js.head()

,id,operation,symbol,pindex,pvalue,value,visit_id,crawl_id,top_url,public_suffix,url
0,1,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/?gws_rd=ssl
1,2,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/?gws_rd=ssl
2,3,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/?gws_rd=ssl
3,4,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/?gws_rd=ssl
4,5,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/?gws_rd=ssl


In [5]:
js[js.operation == "call"].symbol.value_counts().head(15)

CanvasRenderingContext2D.setTransform            1361759
CanvasRenderingContext2D.closePath                387245
CanvasRenderingContext2D.beginPath                274705
CanvasRenderingContext2D.clearRect                255852
window.Storage.getItem                            245579
CanvasRenderingContext2D.fill                     239270
CanvasRenderingContext2D.save                     223513
CanvasRenderingContext2D.restore                  221344
window.Storage.setItem                            212565
HTMLCanvasElement.getContext                      151449
CanvasRenderingContext2D.fillRect                 129560
CanvasRenderingContext2D.createLinearGradient      87621
CanvasRenderingContext2D.fillText                  75467
CanvasRenderingContext2D.arc                       67729
CanvasRenderingContext2D.bezierCurveTo             64692
Name: symbol, dtype: int64

<b>Criteria 1 from Englehardt & Narayanan, 2016:</b>

To be a fingerprinting candidate: "1.The canvas element’s height and width properties must not be set below 16 px."

In [10]:
def width_height_finder(data):
    w = re.compile('[^(line)]width', re.IGNORECASE)
    h = re.compile('[^(line)]height', re.IGNORECASE)
    ww = w.search(data)
    hh = h.search(data)
    if ww!= None:
        return data
    elif hh!=None:
        return data

In [13]:
l = list(map(width_height_finder, list(js[js.operation == "set"].symbol)))
l = [x for x in l if x is not None]

In [14]:
set(l)

{'HTMLCanvasElement.height', 'HTMLCanvasElement.width'}

In [23]:
MIN_CANVAS_IMAGE_WIDTH = 16
MIN_CANVAS_IMAGE_HEIGHT = 16


canvas_size_px = str(js[js.symbol.isin(["HTMLCanvasElement.width", "HTMLCanvasElement.height"])].value)
canvas_size = pd.Series(re.findall(r'\d+', canvas_size_px)).astype(float)
index_size_filter = canvas_size.where(canvas_size > MIN_CANVAS_IMAGE_HEIGHT).dropna().index

In [24]:
index_size_filter

Int64Index([  0,   2,   4,   6,   8,  10,  12,  14,  16,  17,
            ...
            111, 112, 113, 114, 115, 116, 117, 118, 119, 120],
           dtype='int64', length=103)

In [81]:
js.iloc[index_size_filter]

,id,operation,symbol,pindex,pvalue,value,visit_id,crawl_id,top_url,public_suffix,url
0,1,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/?gws_rd=ssl
2,3,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/?gws_rd=ssl
4,5,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/?gws_rd=ssl
6,7,get,window.navigator.plugins[0].description,NaN,NaN,Shockwave Flash 11.2 r202,1,1,http://google.com,NaN,https://www.google.com/?gws_rd=ssl
8,9,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/?gws_rd=ssl
10,11,get,window.sessionStorage,NaN,NaN,{},1,1,http://google.com,NaN,https://www.google.com/xjs/_/js/k=xjs.s.en_US....
12,13,call,window.Storage.setItem,1.0,1,NaN,1,1,http://google.com,NaN,https://www.google.com/xjs/_/js/k=xjs.s.en_US....
14,15,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/xjs/_/js/k=xjs.s.en_US....
16,17,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/xjs/_/js/k=xjs.s.en_US....
17,18,get,window.navigator.userAgent,NaN,NaN,Mozilla/5.0 (X11; Linux x86_64; rv:41.0) Gecko...,1,1,http://google.com,NaN,https://www.google.com/xjs/_/js/k=xjs.s.en_US....


<b> Criteria 2 from Englehardt & Narayanan, 2016: </b> 

To be a fingerprinting candidate: "2.Text must be written to canvas with least two colors or
at least 10 distinct characters."

In [ ]:
#TODO: pindex, pvalue to arguments

In [31]:
def count_distinct_letters(text):
    if text!=None:
        if not isinstance(text, int):
            return len(set(text))

In [32]:
CANVAS_WRITE_FUNCS = [
    "CanvasRenderingContext2D.fillText",
    "CanvasRenderingContext2D.strokeText",
    "CanvasRenderingContext2D.fill"
    ]

In [39]:
text = js[js.symbol.isin(CANVAS_WRITE_FUNCS) & (js.pindex == 0)].pvalue.dropna()
indexx = text.index
text = map(count_distinct_letters, text) 
res = pd.Series(text, index= indexx)
index_count_filter = res.where(res>=10).dropna().index

In [40]:
# js.iloc[index_count_filter]

,id,operation,symbol,pindex,pvalue,value,visit_id,crawl_id,top_url,public_suffix,url
1120,1121,call,CanvasRenderingContext2D.fillText,0.0,Cwm fjordbank glyphs vext quiz,NaN,11,1,http://taobao.com,NaN,http://g.alicdn.com/alilog/mlog/aplus_v2.js
1125,1126,call,CanvasRenderingContext2D.fillText,0.0,Cwm fjordbank glyphs vext quiz,NaN,11,1,http://taobao.com,NaN,http://g.alicdn.com/alilog/mlog/aplus_v2.js
2998,2999,call,CanvasRenderingContext2D.fillText,0.0,Cwm fjordbank glyphs vext quiz,NaN,27,1,http://tmall.com,NaN,https://g.alicdn.com/alilog/mlog/aplus_v2.js
3003,3004,call,CanvasRenderingContext2D.fillText,0.0,Cwm fjordbank glyphs vext quiz,NaN,27,1,http://tmall.com,NaN,https://g.alicdn.com/alilog/mlog/aplus_v2.js
36242,36243,call,CanvasRenderingContext2D.fillText,0.0,fpcf canvasing text,NaN,97,1,http://dropbox.com,NaN,https://cf.dropboxstatic.com/static/javascript...
64743,64744,call,CanvasRenderingContext2D.fillText,0.0,Cwm fjordbank glyphs vext quiz,NaN,135,1,http://cnzz.com,NaN,http://a.tbcdn.cn/s/aplus_v2.js
64748,64749,call,CanvasRenderingContext2D.fillText,0.0,Cwm fjordbank glyphs vext quiz,NaN,135,1,http://cnzz.com,NaN,http://a.tbcdn.cn/s/aplus_v2.js
71113,71114,call,CanvasRenderingContext2D.fillText,0.0,Cwm fjordbank glyphs vext quiz,NaN,152,1,http://detail.tmall.com,NaN,http://g.alicdn.com/alilog/mlog/aplus_v2.js
71118,71119,call,CanvasRenderingContext2D.fillText,0.0,Cwm fjordbank glyphs vext quiz,NaN,152,1,http://detail.tmall.com,NaN,http://g.alicdn.com/alilog/mlog/aplus_v2.js
113102,113103,call,CanvasRenderingContext2D.fillText,0.0,http://valve.github.io,NaN,210,1,http://groupon.com,NaN,https://assets4.grouponcdn.com/layout/assets/s...


<b> Criteria 3 from Englehardt & Narayanan, 2016 </b>

To be a fingerprinting candidate: "3. The script should not call the save, restore, or addEventListener
methods of the rendering context."

In [25]:
CANVAS_FP_DO_NOT_CALL_LIST = ["CanvasRenderingContext2D.save",
                              "CanvasRenderingContext2D.restore",
                              "HTMLCanvasElement.addEventListener"]

In [26]:
index_call_filter = js[~js.symbol.isin(CANVAS_FP_DO_NOT_CALL_LIST)].index
index_call_filter

Int64Index([      0,       1,       2,       3,       4,       5,       6,
                  7,       8,       9,
            ...
            7328705, 7328706, 7328707, 7328708, 7328709, 7328710, 7328711,
            7328712, 7328713, 7328714],
           dtype='int64', length=6881420)

In [43]:
# js.iloc[index_call_filter]

<b>Criteria 4 from Englehardt & Narayanan, 2016: </b>

To be a fingerprinting candidate: "4. The script extracts an image with toDataURL or with a
single call to getImageData that specifies an area with a
minimum size of 16px × 16px"

In [44]:
CANVAS_READ_FUNCS = [
    "HTMLCanvasElement.toDataURL",
    "CanvasRenderingContext2D.getImageData"
    ]

In [74]:
def check_dimensions(image):
    if image!=None:
        img = re.search(r'\d+', image).group(0)
        if float(img)>=16:
            return img

In [77]:
image = js[(js.symbol.isin(CANVAS_READ_FUNCS)) & (js.pindex.isin([2, 3]))].pvalue.astype(str)
indexx = image.index
image = map(check_dimensions, image)
res = pd.Series(image, index = indexx).dropna()
index_image_filter = res.index
index_image_filter

Int64Index([   8343,    8344,    8481,    8482,    8555,    8556,    9209,
               9210,    9246,    9530,
            ...
            7315660, 7315661, 7315703, 7315704, 7325435, 7325436, 7325529,
            7325530, 7325622, 7325623],
           dtype='int64', length=14637)

In [93]:
len(js.iloc[index_image_filter]["visit_id"].value_counts())

3

In [84]:
fp_visit_id = []
for i in set(js.visit_id):
    cond1 = cond2 = cond3 = cond4 = False
    for j in js[js.visit_id==i].index:
        if j in index_size_filter:
            cond1 = True
        if j in index_count_filter:
            cond2 = True
        if j in index_call_filter:
            cond3 = True
        if j in index_image_filter:
            cond4 = True
    if cond1 and cond2 and cond3 and cond4:
        fp_visit_id.append(i)

In [85]:
len(fp_visit_id)

1

In [86]:
lst = []

In [87]:
for i in range(len(js)):
    if i in index_call_filter and i in index_count_filter:
        lst.append(js.iloc[i])
    elif i in index_call_filter and i in index_size_filter:
        lst.append(js.iloc[i])
    elif i in index_call_filter and i in index_image_filter:
        lst.append(js.iloc[i])
    elif i in index_count_filter and i in index_size_filter:
        lst.append(js.iloc[i])
    elif i in index_count_filter and i in index_image_filter:
        lst.append(js.iloc[i])
    elif i in index_image_filter and i in index_size_filter:
        lst.append(js.iloc[i])

In [88]:
df = pd.DataFrame(lst)
len(df)

19918